<a href="https://colab.research.google.com/github/phi1z/1yanagiLab/blob/main/AC_MH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@title File import
import pandas as pd
import numpy as np
import os

# ファイルの読み込み
os.makedirs("data", exist_ok=True)
os.makedirs("result", exist_ok=True)
os.makedirs("raw_result", exist_ok=True)
from IPython.display import clear_output
clear_output()
files = os.listdir("data")
MRIs = [f for f in files if os.path.splitext(f)[1] == '.trc']
MRIs = np.sort(MRIs)
# ファイルの確認
print(f"ファイル数: {len(MRIs)}")
print(f"1: {MRIs[0]} ")
if len(MRIs) > 1:
  print(f"2: {MRIs[1]} ")
else:
  print("......")
if len(MRIs) > 2:
  print(f"3: {MRIs[2]} ")
  print("......")
else:
  print("......")
# indexの表示
data_dir = "data"
rows = []
for filename in os.listdir(data_dir):
    if filename.endswith(".trc"):
        name = os.path.splitext(filename)[0]
        parts = name.split("_")
        parts[2] = int(parts[2].replace("kHz", "")) * 1.0e3
        parts[3] = int(parts[3].replace("mT", ""))
        this_row = [filename, *parts]
        rows.append(this_row)
df = pd.DataFrame(rows, columns=["path", "type", "date", "f", "H", "sample"])
df = df.sort_values(by=["type", "f", "H", "sample"])
df = df.reset_index(drop=True)
display(df[df['type'] == "Mg"])


In [ ]:
#@title Select File

key = 33 #@param {type:"integer"}

MG = df.iloc[key]
condition = [MG['f'], MG['H'], MG['sample']]

print(np.array(condition))

Mg = df[(df['type'] == "Mg") & (df['f'] == condition[0]) &
        (df['H'] == condition[1]) & (df['sample'] == condition[2])]
BG = df[(df['type'] == "BG") & (df['f'] == condition[0]) &
        (df['H'] == condition[1])]
H1 = df[(df['type'] == "H1") & (df['f'] == condition[0]) &
        (df['H'] == condition[1])]
H2 = df[(df['type'] == "H2") & (df['f'] == condition[0]) &
        (df['H'] == condition[1]) & (df['sample'] == condition[2])]
All = pd.concat([BG, H1, H2, Mg])
All = All.reset_index(drop=True)
display(All)


## Step by Step

In [ ]:
#@title Read Signal

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy import integrate

def read_trc(path):
  df = pd.read_csv(path, skiprows=4, sep=",")
  df.columns = ["time", "amp"]
  df['time'] = df['time'] - df['time'][0]
  df = df.dropna().reset_index(drop=True)
  return df

def get_flux(df, ax, sampe=""):
  df['flux'] = np.cumsum(df['amp'] * df['time'].diff())
  df = df.dropna().reset_index(drop=True)
  df.head()

  ax.grid(True)
  ax.set_title(sampe, fontsize=14) # Changed title to set_title
  ax.plot(df['time'], df['amp'], label=r"$d\Phi/dt$", color="blue")
  ax2 = ax.twinx()
  ax2.plot(df['time'], df['flux'], label=r"$\Phi$", color="orange")
  ax.set_xlabel("time [s]", fontsize=14)
  ax.set_ylabel("Voltage [V]", fontsize=14)
  ax2.set_ylabel("Flux [s.V]", fontsize=14)
  # plot legend with one legend
  lines, labels = ax.get_legend_handles_labels()
  lines2, labels2 = ax2.get_legend_handles_labels()
  ax2.legend(lines + lines2, labels + labels2)
  # plt.show() # Removed plt.show() from the function to show all subplots at once
  return df

path_list = []
for path in All['path']:
  path_list.append(f"data/{path}")

BG_df = read_trc(path_list[0])
H1_df = read_trc(path_list[1])
H2_df = read_trc(path_list[2])
Mg_df = read_trc(path_list[3])

fig, axs = plt.subplots(2, 2, figsize=(12, 8))

BG = get_flux(BG_df, axs[0, 0], "BG")
H1 = get_flux(H1_df, axs[0, 1], "H1")
H2 = get_flux(H2_df, axs[1, 0], "H2")
Mg = get_flux(Mg_df, axs[1, 1], "Mg")

plt.tight_layout() # Added for better layout
plt.show()

In [ ]:
#@title liner offset

from matplotlib import axes

def fit_AMF(df, ax, freq, sampe="", p0=[1.0e-7, 1.0, 1.0e3, 1.0e-5]):
  def Liner_AMF(x, H, phi, a, b):
    AMF = H * np.sin(2*np.pi*freq*x + phi)
    liner = a * x + b
    return AMF + liner

  popt, pcov = optimize.curve_fit(Liner_AMF, xdata=df['time'], ydata=df['flux'],
                                  p0=p0)
  err = np.sqrt(np.diag(pcov))
  line_y = popt[2] * df['time'] + popt[3]
  fit_y = df['flux'] - line_y
  balance = (np.max(fit_y) + np.min(fit_y)) / 2
  fit_y = fit_y - balance

  ax.grid(True)
  ax.set_title(sampe, fontsize=14)
  ax.hlines(y=0, xmin=np.min(df['time']), xmax=np.max(df['time']), linestyles='--', colors="orange")
  ax.plot(df['time'], df['flux'], label=r"$\Phi$", color="blue")
  ax.plot(df['time'], fit_y, label=r"AMF fit", color="orange")
  ax.plot(df['time'], line_y, label=r"Linear offset", color="blue", linestyle="--")
  ax.set_xlim(np.min(df['time']), np.max(df['time']))
  ax.set_xlabel("time [s]", fontsize=14)
  ax.set_ylabel("Flux [s.V]", fontsize=14)
  ax.legend(fontsize=14)
  return fit_y, popt, err

f = condition[0]

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

BG['fit_flux'], BG_popt, BG_err = fit_AMF(BG, axes[0, 0], f, "BG", p0=[1.0e-9, 1.0, 1.0e-2, 1.0e-9])
print("============ BG was done ============")
H1['fit_flux'], H1_popt, H1_err = fit_AMF(H1, axes[0, 1], f, "H1")
print("============ H1 was done ============")
H2['fit_flux'], H2_popt, H2_err = fit_AMF(H2, axes[1, 0], f, "H2")
print("============ H2 was done ============")
Mg['fit_flux'], Mg_popt, Mg_err = fit_AMF(Mg, axes[1, 1], f, "Mg")
print("============ Mg was done ============")

plt.tight_layout()
plt.show()


In [ ]:
#@title Syncronize H1 and H2

def sync_phase(h1, h2, p1, p2, rate=1.0):
  phi1 = p1[1]
  phi2 = p2[1]
  dphi = phi1 - phi2
  print(f"dphi = {dphi*180:.3g}deg.")
  if dphi > np.pi:
    dphi = dphi - 2*np.pi
  elif dphi < -1 * 2*np.pi:
    dphi = dphi + 2*np.pi
  print(f"re_dphi = {dphi*180:.3g}deg.")
  del_t = np.abs(dphi) / (2 * np.pi * f) * rate
  print(f"del_t = {del_t:.3g} [s]")
  if dphi >= 0:
    n = len(h1)
    h2 = h2[h2['time'] >= del_t].copy()
    h2['time'] = h2['time'] - del_t
    h1 = h1.iloc[:(len(h2))]
    n_cut = n - len(h2)
    n_cut = -1 * n_cut
    h1 = h1.reset_index(drop=True)
    h2 = h2.reset_index(drop=True)
  else:
    n = len(h2)
    h1 = h1[h1['time'] >= del_t].copy()
    h1['time'] = h1['time'] - del_t
    h2 = h2.iloc[:(len(h1))]
    n_cut = n - len(h1)
    h1 = h1.reset_index(drop=True)
    h2 = h2.reset_index(drop=True)

  print(n_cut)
  dif = h2['fit_flux'] - h1['fit_flux']
  max_h1 = np.max(h1['fit_flux'])
  max_h2 = np.max(h2['fit_flux'])
  max_h = np.max([max_h1, max_h2])
  max_dif = np.max(np.abs(dif))
  print(f"diff = {max_dif/max_h * 100:.3f}%")
  fig, ax = plt.subplots(figsize=(8, 6))
  ax2 = ax.twinx()
  ax.grid(True)
  ax2.plot(h1['time'], dif, label=r"H2 - H1", color="black")
  ax.plot(h1['time'], h1['fit_flux'], label=r"H1", linewidth=3)
  ax.plot(h2['time'], h2['fit_flux'], label=r"H2", linestyle="--", linewidth=3)
  ax.set_xlabel("time [s]")
  ax.set_ylabel("Flux [s.V]")
  ax2.set_ylabel("Flux [s.V]")
  ax.legend()
  ax2.legend(loc=2)
  plt.show()
  return h1, h2, n_cut

dif_time_rate = 1.0 #@param {type:"number"}

H1_sync, H2_sync, n_dif = sync_phase(H1, H2, H1_popt, H2_popt, rate=dif_time_rate)

In [ ]:
#@title plot M-H

if n_dif > 0:
  Mg_sync = Mg.iloc[:(len(Mg) - n_dif)].copy()
  Mg_sync = Mg_sync.reset_index(drop=True)
  Mg_sync['fit_flux'] = Mg_sync['fit_flux'] - BG[n_dif:]['fit_flux']
else:
  n_dif = -1 * n_dif
  Mg_sync = Mg.iloc[n_dif:].copy()
  Mg_sync['time'] = Mg_sync['time'] - Mg_sync['time'].iloc[0]
  Mg_sync = Mg_sync.reset_index(drop=True)
  Mg_sync['fit_flux'] = Mg_sync['fit_flux'] - BG[:(len(Mg) - n_dif)]['fit_flux']

Field_rate = 30969715 # Vs to mT
Magnetization_rate = 1035712# Vs to emu

H2_sync['value (mT)'] = H2_sync['fit_flux'] * Field_rate
Mg_sync['value (emu)'] = Mg_sync['fit_flux'] * Magnetization_rate

T = H1_sync['time']
T = T - T[0]
H = H2_sync['fit_flux']
H = H * Field_rate
M = Mg_sync['fit_flux']
M = M * Magnetization_rate

out_df = pd.DataFrame({"T": T, "H": H, "M": M})
filename = f"{condition[0]/1.0e3:.0f}kHz_{condition[1]}mT_{condition[2]}.csv"
out_df.to_csv(os.path.join("result/", filename), index=False)
H2_sync.to_csv(os.path.join("raw_result/", "H2_" + filename), index=False)
Mg_sync.to_csv(os.path.join("raw_result/", "Mg_" + filename), index=False)

Mass = 9.82 #@param {type:"number"}
Density = 5.17 #@param {type:"number"}
Unit = "SI" #@param ["cgs", "SI"]

def plot_MH(df, mass=10, d=1, unit="cgs"):
  fig, ax = plt.subplots(figsize=(8, 6))
  H_mT = df['H']
  M_emu = df['M']
  mass_g = mass * 1.0e-3
  if unit == "cgs":
    X = H_mT * 10
    Y = M_emu / mass
    X_unit = "Oe"
    Y_unit = "emu/g"
  elif unit == "SI":
    X = H_mT * 10 / (4*np.pi)
    Y = M_emu / mass_g * d
    X_unit = "kA/m"
    Y_unit = "kA/m"
  else:
    print("Unit error")
    return 0

  ax.grid(True)
  ax.plot(X, Y, label=condition[2],)

  ax.set_xlabel(f"Magnetic Field [{X_unit}]", fontsize=16)
  ax.set_ylabel(f"Magnetization [{Y_unit}]", fontsize=16)
  ax.legend(fontsize=16)
  plt.show()

plot_MH(out_df, mass=Mass, unit=Unit)


## At Once

Please run all cell in "**Display Sample List**" in advance!!

In [ ]:
#@title Display Sample List


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy import integrate

def read_trc(path):
  df = pd.read_csv(path, skiprows=4, sep=",")
  df.columns = ["time", "amp"]
  df['time'] = df['time'] - df['time'][0]
  df = df.dropna().reset_index(drop=True)
  return df

def get_flux(df, ax, sampe=""):
  df['flux'] = np.cumsum(df['amp'] * df['time'].diff())
  df = df.dropna().reset_index(drop=True)
  df.head()

  ax.grid(True)
  ax.set_title(sampe, fontsize=14) # Changed title to set_title
  ax.plot(df['time'], df['amp'], label=r"$d\Phi/dt$", color="blue")
  ax2 = ax.twinx()
  ax2.plot(df['time'], df['flux'], label=r"$\Phi$", color="orange")
  ax.set_xlabel("time [s]", fontsize=14)
  ax.set_ylabel("Voltage [V]", fontsize=14)
  ax2.set_ylabel("Flux [s.V]", fontsize=14)
  # plot legend with one legend
  lines, labels = ax.get_legend_handles_labels()
  lines2, labels2 = ax2.get_legend_handles_labels()
  ax2.legend(lines + lines2, labels + labels2)
  # plt.show() # Removed plt.show() from the function to show all subplots at once
  return df


from matplotlib import axes

def fit_AMF(df, ax, freq, sampe="", p0=[1.0e-7, 1.0, 1.0e3, 1.0e-5]):
  def Liner_AMF(x, H, phi, a, b):
    AMF = H * np.sin(2*np.pi*freq*x + phi)
    liner = a * x + b
    return AMF + liner

  popt, pcov = optimize.curve_fit(Liner_AMF, xdata=df['time'], ydata=df['flux'],
                                  p0=p0)
  err = np.sqrt(np.diag(pcov))
  line_y = popt[2] * df['time'] + popt[3]
  fit_y = df['flux'] - line_y
  balance = (np.max(fit_y) + np.min(fit_y)) / 2
  fit_y = fit_y - balance

  ax.grid(True)
  ax.set_title(sampe, fontsize=14)
  ax.hlines(y=0, xmin=np.min(df['time']), xmax=np.max(df['time']), linestyles='--', colors="orange")
  ax.plot(df['time'], df['flux'], label=r"$\Phi$", color="blue")
  ax.plot(df['time'], fit_y, label=r"AMF fit", color="orange")
  ax.plot(df['time'], line_y, label=r"Linear offset", color="blue", linestyle="--")
  ax.set_xlim(np.min(df['time']), np.max(df['time']))
  ax.set_xlabel("time [s]", fontsize=14)
  ax.set_ylabel("Flux [s.V]", fontsize=14)
  ax.legend(fontsize=14)
  return fit_y, popt, err


def sync_phase(h1, h2, p1, p2, rate=1.0):
  phi1 = p1[1]
  phi2 = p2[1]
  dphi = phi1 - phi2
  print(f"dphi = {dphi*180:.3g}deg.")
  if dphi > np.pi:
    dphi = dphi - 2*np.pi
  elif dphi < -1 * 2*np.pi:
    dphi = dphi + 2*np.pi
  print(f"re_dphi = {dphi*180:.3g}deg.")
  del_t = np.abs(dphi) / (2 * np.pi * f) * rate
  print(f"del_t = {del_t:.3g} [s]")
  if dphi >= 0:
    n = len(h1)
    h2 = h2[h2['time'] >= del_t].copy()
    h2['time'] = h2['time'] - del_t
    h1 = h1.iloc[:(len(h2))]
    n_cut = n - len(h2)
    n_cut = -1 * n_cut
    h1 = h1.reset_index(drop=True)
    h2 = h2.reset_index(drop=True)
  else:
    n = len(h2)
    h1 = h1[h1['time'] >= del_t].copy()
    h1['time'] = h1['time'] - del_t
    h2 = h2.iloc[:(len(h1))]
    n_cut = n - len(h1)
    h1 = h1.reset_index(drop=True)
    h2 = h2.reset_index(drop=True)

  print(n_cut)
  dif = h2['fit_flux'] - h1['fit_flux']
  max_h1 = np.max(h1['fit_flux'])
  max_h2 = np.max(h2['fit_flux'])
  max_h = np.max([max_h1, max_h2])
  max_dif = np.max(np.abs(dif))
  print(f"diff = {max_dif/max_h * 100:.3f}%")
  fig, ax = plt.subplots(figsize=(8, 6))
  ax2 = ax.twinx()
  ax.grid(True)
  ax2.plot(h1['time'], dif, label=r"H2 - H1", color="black")
  ax.plot(h1['time'], h1['fit_flux'], label=r"H1", linewidth=3)
  ax.plot(h2['time'], h2['fit_flux'], label=r"H2", linestyle="--", linewidth=3)
  ax.set_xlabel("time [s]")
  ax.set_ylabel("Flux [s.V]")
  ax2.set_ylabel("Flux [s.V]")
  ax.legend()
  ax2.legend(loc=2)
  plt.show()
  return h1, h2, n_cut


def plot_MH(df, mass=10, d=1, unit="cgs"):
  fig, ax = plt.subplots(figsize=(8, 6))
  H_mT = df['H']
  M_emu = df['M']
  mass_g = mass * 1.0e-3
  if unit == "cgs":
    X = H_mT * 10
    Y = M_emu / mass
    X_unit = "Oe"
    Y_unit = "emu/g"
  elif unit == "SI":
    X = H_mT * 10 / (4*np.pi)
    Y = M_emu / mass_g * d
    X_unit = "kA/m"
    Y_unit = "kA/m"
  else:
    print("Unit error")
    return 0

  ax.grid(True)
  ax.plot(X, Y, label=condition[2],)

  ax.set_xlabel(f"Magnetic Field [{X_unit}]", fontsize=16)
  ax.set_ylabel(f"Magnetization [{Y_unit}]", fontsize=16)
  ax.legend(fontsize=16)
  plt.show()


display(df[df['type'] == "Mg"])

In [ ]:
#@title At Once

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy import integrate

# ====================================

key = 33 #@param {type:"integer"}

MG = df.iloc[key]
condition = [MG['f'], MG['H'], MG['sample']]

print(f"sample: {condition[2]}, f = {condition[0]/1.0e3:.0f} kHz, Amp. = {condition[1]:.0f} mT")

Mg = df[(df['type'] == "Mg") & (df['f'] == condition[0]) &
        (df['H'] == condition[1]) & (df['sample'] == condition[2])]
BG = df[(df['type'] == "BG") & (df['f'] == condition[0]) &
        (df['H'] == condition[1])]
H1 = df[(df['type'] == "H1") & (df['f'] == condition[0]) &
        (df['H'] == condition[1])]
H2 = df[(df['type'] == "H2") & (df['f'] == condition[0]) &
        (df['H'] == condition[1]) & (df['sample'] == condition[2])]
All = pd.concat([BG, H1, H2, Mg])
All = All.reset_index(drop=True)
display(All)

# ====================================

path_list = []
for path in All['path']:
  path_list.append(f"data/{path}")

BG_df = read_trc(path_list[0])
H1_df = read_trc(path_list[1])
H2_df = read_trc(path_list[2])
Mg_df = read_trc(path_list[3])

fig, axs = plt.subplots(2, 2, figsize=(12, 8))

BG = get_flux(BG_df, axs[0, 0], "BG")
H1 = get_flux(H1_df, axs[0, 1], "H1")
H2 = get_flux(H2_df, axs[1, 0], "H2")
Mg = get_flux(Mg_df, axs[1, 1], "Mg")

plt.tight_layout() # Added for better layout
plt.show()

# ====================================

f = condition[0]

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

BG['fit_flux'], BG_popt, BG_err = fit_AMF(BG, axes[0, 0], f, "BG", p0=[1.0e-9, 1.0, 1.0e-2, 1.0e-9])
print("============ BG was done ============")
H1['fit_flux'], H1_popt, H1_err = fit_AMF(H1, axes[0, 1], f, "H1")
print("============ H1 was done ============")
H2['fit_flux'], H2_popt, H2_err = fit_AMF(H2, axes[1, 0], f, "H2")
print("============ H2 was done ============")
Mg['fit_flux'], Mg_popt, Mg_err = fit_AMF(Mg, axes[1, 1], f, "Mg")
print("============ Mg was done ============")

plt.tight_layout()
plt.show()

# ====================================

dif_time_rate = 1.0 #@param {type:"number"}

H1_sync, H2_sync, n_dif = sync_phase(H1, H2, H1_popt, H2_popt, rate=dif_time_rate)

# ====================================

if n_dif > 0:
  Mg_sync = Mg.iloc[:(len(Mg) - n_dif)].copy()
  Mg_sync = Mg_sync.reset_index(drop=True)
  Mg_sync['fit_flux'] = Mg_sync['fit_flux'] - BG[n_dif:]['fit_flux']
else:
  n_dif = -1 * n_dif
  Mg_sync = Mg.iloc[n_dif:].copy()
  Mg_sync['time'] = Mg_sync['time'] - Mg_sync['time'].iloc[0]
  Mg_sync = Mg_sync.reset_index(drop=True)
  Mg_sync['fit_flux'] = Mg_sync['fit_flux'] - BG[:(len(Mg) - n_dif)]['fit_flux']

Field_rate = 30969715 # Vs to mT
Magnetization_rate = 1035712# Vs to emu

H2_sync['value (mT)'] = H2_sync['fit_flux'] * Field_rate
Mg_sync['value (emu)'] = Mg_sync['fit_flux'] * Magnetization_rate

T = H1_sync['time']
T = T - T[0]
H = H2_sync['fit_flux']
H = H * Field_rate
M = Mg_sync['fit_flux']
M = M * Magnetization_rate

out_df = pd.DataFrame({"T": T, "H": H, "M": M})
filename = f"{condition[0]/1.0e3:.0f}kHz_{condition[1]}mT_{condition[2]}.csv"
out_df.to_csv(os.path.join("result/", filename), index=False)
H2_sync.to_csv(os.path.join("raw_result/", "H2_" + filename), index=False)
Mg_sync.to_csv(os.path.join("raw_result/", "Mg_" + filename), index=False)

Mass = 9.82 #@param {type:"number"}
Density = 5.17 #@param {type:"number"}
Unit = "SI" #@param ["cgs", "SI"]

plot_MH(out_df, mass=Mass, unit=Unit)


# ====================================



In [ ]:
#@title At Once  (at first, Mg - BG)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy import integrate

# ====================================

key = 33 #@param {type:"integer"}

MG = df.iloc[key]
condition = [MG['f'], MG['H'], MG['sample']]

print(f"sample: {condition[2]}, f = {condition[0]/1.0e3:.0f} kHz, Amp. = {condition[1]:.0f} mT")

Mg = df[(df['type'] == "Mg") & (df['f'] == condition[0]) &
        (df['H'] == condition[1]) & (df['sample'] == condition[2])]
BG = df[(df['type'] == "BG") & (df['f'] == condition[0]) &
        (df['H'] == condition[1])]
H1 = df[(df['type'] == "H1") & (df['f'] == condition[0]) &
        (df['H'] == condition[1])]
H2 = df[(df['type'] == "H2") & (df['f'] == condition[0]) &
        (df['H'] == condition[1]) & (df['sample'] == condition[2])]
All = pd.concat([BG, H1, H2, Mg])
All = All.reset_index(drop=True)
display(All)

# ====================================

path_list = []
for path in All['path']:
  path_list.append(f"data/{path}")

BG_df = read_trc(path_list[0])
H1_df = read_trc(path_list[1])
H2_df = read_trc(path_list[2])
Mg_df = read_trc(path_list[3])

fig, axs = plt.subplots(2, 2, figsize=(12, 8))

BG = get_flux(BG_df, axs[0, 0], "BG")
H1 = get_flux(H1_df, axs[0, 1], "H1")
H2 = get_flux(H2_df, axs[1, 0], "H2")
Mg = get_flux(Mg_df, axs[1, 1], "Mg")

plt.tight_layout() # Added for better layout
plt.show()

# ====================================

f = condition[0]

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

BG['fit_flux'], BG_popt, BG_err = fit_AMF(BG, axes[0, 0], f, "BG", p0=[1.0e-9, 1.0, 1.0e-2, 1.0e-9])
print("============ BG was done ============")
H1['fit_flux'], H1_popt, H1_err = fit_AMF(H1, axes[0, 1], f, "H1")
print("============ H1 was done ============")
H2['fit_flux'], H2_popt, H2_err = fit_AMF(H2, axes[1, 0], f, "H2")
print("============ H2 was done ============")
Mg['fit_flux'], Mg_popt, Mg_err = fit_AMF(Mg, axes[1, 1], f, "Mg")
print("============ Mg was done ============")

plt.tight_layout()
plt.show()

# ====================================

dif_time_rate = 1.0 #@param {type:"number"}

# Mg - BG
Mg['fit_flux'] = Mg['fit_flux'] - BG['fit_flux']

H1_sync, H2_sync, n_dif = sync_phase(H1, H2, H1_popt, H2_popt, rate=dif_time_rate)

# ====================================

if n_dif > 0:
  Mg_sync = Mg.iloc[:(len(Mg) - n_dif)].copy()
  Mg_sync = Mg_sync.reset_index(drop=True)
else:
  n_dif = -1 * n_dif
  Mg_sync = Mg.iloc[n_dif:].copy()
  Mg_sync['time'] = Mg_sync['time'] - Mg_sync['time'].iloc[0]
  Mg_sync = Mg_sync.reset_index(drop=True)


Field_rate = 30969715 # Vs to mT
Magnetization_rate = 1035712# Vs to emu

H2_sync['value (mT)'] = H2_sync['fit_flux'] * Field_rate
Mg_sync['value (emu)'] = Mg_sync['fit_flux'] * Magnetization_rate

T = H1_sync['time']
T = T - T[0]
H = H2_sync['fit_flux']
H = H * Field_rate
M = Mg_sync['fit_flux']
M = M * Magnetization_rate

out_df = pd.DataFrame({"T": T, "H": H, "M": M})
filename = f"{condition[0]/1.0e3:.0f}kHz_{condition[1]}mT_{condition[2]}.csv"
out_df.to_csv(os.path.join("result/", filename), index=False)
H2_sync.to_csv(os.path.join("raw_result/", "H2_" + filename), index=False)
Mg_sync.to_csv(os.path.join("raw_result/", "Mg_" + filename), index=False)

Mass = 9.82 #@param {type:"number"}
Density = 5.17 #@param {type:"number"}
Unit = "SI" #@param ["cgs", "SI"]

def plot_MH(df, mass=1.0e-3, d=1, unit="cgs"):
  fig, ax = plt.subplots(figsize=(8, 6))
  H_mT = df['H']
  M_emu = df['M']
  mass_g = mass * 1.0e-3
  if unit == "cgs":
    X = H_mT * 10
    Y = M_emu / mass
    X_unit = "Oe"
    Y_unit = "emu/g"
  elif unit == "SI":
    X = H_mT * 10 / (4*np.pi)
    Y = M_emu / mass_g * d
    X_unit = "kA/m"
    Y_unit = "kA/m"
  else:
    print("Unit error")
    return 0

  ax.grid(True)
  ax.plot(X, Y, label=condition[2],)

  ax.set_xlabel(f"Magnetic Field [{X_unit}]", fontsize=16)
  ax.set_ylabel(f"Magnetization [{Y_unit}]", fontsize=16)
  ax.legend(fontsize=16)
  plt.show()

plot_MH(out_df, mass=Mass, unit=Unit)

# ====================================



## Save

In [ ]:
#@title Zip results

!zip -r result.zip result
!zip -r raw_result.zip raw_result